# WIP # Chapter 2:

**Comprehensive Evaluation Strategies**

In [ ]:
import os
import difflib
import Levenshtein
from nltk import word_tokenize
from nltk.translate import meteor
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from ranx import Qrels, Run, evaluate
from rouge import Rouge
from sklearn.metrics.pairwise import cosine_similarity
import wandb
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
import pandas as pd
import json
import pathlib
from typing import List, Dict, Any
from tqdm import tqdm
import cohere
import nest_asyncio
nest_asyncio.apply()
import asyncio
from dotenv import load_dotenv
import numpy as np

load_dotenv()


## Building and improving an evaluation dataset

### Collecting data for evaluation
Get from data from the docs website [FAQs](https://docs.wandb.ai/guides/technical-faq) to test the system.

In [ ]:
# from datetime import datetime

# # TODO: Remove this once we more to the final project
# eval_artifact = wandb.Artifact(
#     name="eval_dataset",
#     type="dataset",
#     description="Evaluation dataset for RAG",
#     metadata={
#         "total_samples": 20,
#         "date_collected": datetime.now().strftime("%Y-%m-%d"),
#         "chapter": "Chapter 1",
#     },
# )
# eval_artifact.add_file("../data/eval/eval_dataset.jsonl")
# run.log_artifact(eval_artifact)

In [ ]:
WANDB_ENTITY = "rag-course"
WANDB_PROJECT = "dev"

wandb.require("core")

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    group="Chapter 2",
)

In [ ]:
eval_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/eval_dataset:latest", type="dataset"
)
eval_dir = eval_artifact.download("../data/eval")
eval_dataset = pd.read_json(
    f"{eval_dir}/final_eval_dataset.jsonl", lines=True, orient="records"
)
eval_samples = eval_dataset.to_dict(orient="records")
eval_dataset

### Evaluating the Retriever

This is a search problem, it's easiest to start with tradiaional Information retrieval metrics.


ref: https://weaviate.io/blog/retrieval-evaluation-metrics

**TODO** Add weave model and evals in this section

In [ ]:
# Reload the data from Chapter 1
chunked_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/chunked_data:latest", type="dataset"
)
artifact_dir = chunked_artifact.download()
chunked_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
chunked_data = list(map(json.loads, chunked_data_file.read_text().splitlines()))
chunked_data[:2]

In [ ]:
import weave

weave_client = weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

In [ ]:
# %load scripts/retriever
import weave
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist

class Retriever(weave.Model):
    vectorizer: TfidfVectorizer = TfidfVectorizer()
    index: list = None
    data: list = None

    @weave.op()
    def index_data(self, data):
        self.data = data
        docs = [doc["cleaned_content"] for doc in data]
        self.index = self.vectorizer.fit_transform(docs)

    @weave.op()
    def search(self, query, k=5):
        query_vec = self.vectorizer.transform([query])
        cosine_distances = cdist(
            query_vec.todense(), self.index.todense(), metric="cosine"
        )[0]
        top_k_indices = cosine_distances.argsort()[:k]
        output = []
        for idx in top_k_indices:
            output.append(
                {
                    "source": self.data[idx]["metadata"]["source"],
                    "text": self.data[idx]["cleaned_content"],
                    "score": 1 - cosine_distances[idx],
                }
            )
        return output

    @weave.op()
    def predict(self, query: str, k: int):
        return self.search(query, k)


In [ ]:
retriever = Retriever()
retriever.index_data(chunked_data)
retriever.predict("what is wandb", 5)

In [ ]:
eval_samples[0]

In [ ]:
# %load scripts/retrieval_metrics.py
import weave
import numpy as np
from typing import List, Dict, Any


@weave.op()
def compute_hit_rate(
    model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]
) -> float:
    """
    Calculate the hit rate (precision) for a single query.

    Args:
        model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.
            Each dictionary contains:
                - 'source': A unique identifier for the document.
                - 'score': The relevance score of the document.
        contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.
            Each dictionary contains:
                - 'source': A unique identifier for the relevant document.

    Returns:
        float: The hit rate (precision).

    The hit rate (precision) measures the proportion of retrieved documents that are relevant.
    It is calculated using the following formula:

    \[ \text{Hit Rate (Precision)} = \frac{\text{Number of Relevant Documents Retrieved}}{\text{Total Number of Documents Retrieved}} \]

    This metric is useful for assessing the accuracy of the retrieval system by determining the relevance of the retrieved documents.
    ```
    """
    search_results = [doc["source"] for doc in model_output]
    relevant_sources = [
        context["source"] for context in contexts if context["relevance"] != 0
    ]

    # Calculate the number of relevant documents retrieved
    relevant_retrieved = sum(
        1 for source in search_results if source in relevant_sources
    )

    # Calculate the hit rate (precision)
    hit_rate = relevant_retrieved / len(search_results) if search_results else 0.0

    return hit_rate


@weave.op
def compute_mrr(
    model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]
) -> float:
    """
    Calculate the Mean Reciprocal Rank (MRR) for a single query.

    Args:
        model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.
            Each dictionary contains:
                - 'source': A unique identifier for the document.
                - 'score': The relevance score of the document.
        contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.
            Each dictionary contains:
                - 'source': A unique identifier for the relevant document.

    Returns:
        float: The MRR score for the given query.

    MRR measures the rank of the first relevant document in the result list.
    It is calculated using the following formula:

    \[ \text{MRR} = \frac{1}{\text{rank of first relevant document}} \]

    If no relevant document is found, MRR is 0.

    This metric is useful for evaluating systems where there is typically one relevant document
    and the user is interested in finding that document quickly.
    """
    relevant_sources = [
        context["source"] for context in contexts if context["relevance"] != 0
    ]

    mrr_score = 0
    for rank, result in enumerate(model_output, 1):
        if result["source"] in relevant_sources:
            mrr_score = 1 / rank
            break
    return mrr_score


# NDCG (Normalized Discounted Cumulative Gain)
@weave.op
def compute_ndcg(
    model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]
) -> float:
    """
    Calculate the Normalized Discounted Cumulative Gain (NDCG) for a single query.

    Args:
        model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.
            Each dictionary contains:
                - 'source': A unique identifier for the document.
                - 'score': The cosine similarity score of the document to the query.
        contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.
            Each dictionary contains:
                - 'source': A unique identifier for the relevant document.

    Returns:
        float: The NDCG score for the given query.

    NDCG measures the ranking quality of the search results, taking into account the position of relevant documents.

    NDCG Formula:
    1. Calculate the Discounted Cumulative Gain (DCG):
       \[ \text{DCG}_p = \sum_{i=1}^p \frac{2^{rel_i} - 1}{\log_2(i + 1)} \]
       where \( rel_i \) is the relevance score of the document at position \( i \).

    2. Calculate the Ideal Discounted Cumulative Gain (IDCG), which is the DCG of the ideal ranking:
       \[ \text{IDCG}_p = \sum_{i=1}^p \frac{2^{rel_i} - 1}{\log_2(i + 1)} \]
       where documents are sorted by their relevance scores in descending order.

    3. Normalize the DCG by dividing it by the IDCG to get NDCG:
       \[ \text{NDCG}_p = \frac{\text{DCG}_p}{\text{IDCG}_p} \]

    This implementation uses continuous relevance scores.
    """
    relevant_sources = {
        context["source"] for context in contexts if context["relevance"] != 0
    }

    dcg = 0.0
    idcg = 0.0

    # Calculate DCG
    for i, result in enumerate(model_output):
        if result["source"] in relevant_sources:
            dcg += (2 ** result["score"] - 1) / np.log2(
                i + 2
            )  # i+2 because log2 starts at 1 for i=0

    # Sort the results by score to calculate IDCG
    sorted_model_output = sorted(model_output, key=lambda x: x["score"], reverse=True)

    # Calculate IDCG
    for i, result in enumerate(sorted_model_output):
        if result["source"] in relevant_sources:
            idcg += (2 ** result["score"] - 1) / np.log2(i + 2)

    # To avoid division by zero
    if idcg == 0:
        return 0.0

    # Calculate nDCG
    ndcg = dcg / idcg
    return ndcg


# MAP (Mean Average Precision)
@weave.op()
def compute_map(
    model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]
) -> float:
    """
    Calculate the Mean Average Precision (MAP) for a single query.

    Args:
        model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.
            Each dictionary contains:
                - 'source': A unique identifier for the document.
                - 'score': The relevance score of the document.
        contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.
            Each dictionary contains:
                - 'source': A unique identifier for the relevant document.

    Returns:
        float: The MAP score for the given query.

    MAP provides a single-figure measure of quality across recall levels.
    For a single query, it's equivalent to the Average Precision (AP).
    It's calculated using the following formula:

    \[ \text{MAP} = \frac{\sum_{k=1}^n P(k) \times \text{rel}(k)}{\text{number of relevant documents}} \]

    Where:
    - n is the number of retrieved documents
    - P(k) is the precision at cut-off k in the list
    - rel(k) is an indicator function: 1 if the item at rank k is relevant, 0 otherwise
    MAP considers both precision and recall, as well as the ranking of relevant documents.

    """
    relevant_sources = {
        context["source"] for context in contexts if context["relevance"] != 0
    }

    num_relevant = 0
    sum_precision = 0.0

    for i, result in enumerate(model_output):
        if result["source"] in relevant_sources:
            num_relevant += 1
            sum_precision += num_relevant / (i + 1)

    if num_relevant == 0:
        return 0.0

    average_precision = sum_precision / len(relevant_sources)
    return average_precision


@weave.op()
def compute_precision(
    model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]
) -> float:
    """
    Calculate the Precision for a single query.

    Args:
        model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.
            Each dictionary contains:
                - 'source': A unique identifier for the document.
                - 'score': The relevance score of the document.
        contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.
            Each dictionary contains:
                - 'source': A unique identifier for the relevant document.

    Returns:
        float: The Precision score for the given query.

    Precision measures the proportion of retrieved documents that are relevant.
    It is calculated using the following formula:

    \[ \text{Precision} = \frac{\text{Number of Relevant Documents Retrieved}}{\text{Total Number of Documents Retrieved}} \]
    """
    relevant_sources = {
        context["source"] for context in contexts if context["relevance"] != 0
    }
    retrieved_sources = {result["source"] for result in model_output}

    relevant_retrieved = relevant_sources & retrieved_sources

    precision = (
        len(relevant_retrieved) / len(retrieved_sources) if retrieved_sources else 0.0
    )
    return precision


# Recall
@weave.op()
def compute_recall(
    model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]
) -> float:
    """
    Calculate the Recall for a single query.

    Args:
        model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.
            Each dictionary contains:
                - 'source': A unique identifier for the document.
                - 'score': The relevance score of the document.
        contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.
            Each dictionary contains:
                - 'source': A unique identifier for the relevant document.

    Returns:
        float: The Recall score for the given query.

    Recall measures the proportion of relevant documents that are retrieved.
    It is calculated using the following formula:

    \[ \text{Recall} = \frac{\text{Number of Relevant Documents Retrieved}}{\text{Total Number of Relevant Documents}} \]
    """
    relevant_sources = {
        context["source"] for context in contexts if context["relevance"] != 0
    }
    retrieved_sources = {result["source"] for result in model_output}

    relevant_retrieved = relevant_sources & retrieved_sources

    recall = (
        len(relevant_retrieved) / len(relevant_sources) if relevant_sources else 0.0
    )
    return recall


# F1 Score
@weave.op()
def compute_f1_score(
    model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]
) -> float:
    """
    Calculate the F1-Score for a single query.

    Args:
        model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.
            Each dictionary contains:
                - 'source': A unique identifier for the document.
                - 'score': The relevance score of the document.
        contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.
            Each dictionary contains:
                - 'source': A unique identifier for the relevant document.

    Returns:
        float: The F1-Score for the given query.

    F1-Score is the harmonic mean of Precision and Recall.
    It is calculated using the following formula:

    \[ \text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} \]
    """
    precision = compute_precision(model_output, contexts)
    recall = compute_recall(model_output, contexts)

    if precision + recall == 0:
        return 0.0

    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score


In [ ]:
## Here's how we can evaluate the retrieval system in python

# hit_rates = []
# for sample in eval_samples:
#     query = sample["question"]
#     expected_source = sample["source"]
#     search_results = [doc['source'] for doc in retriever.search(query, k=5)]
#     hit_rate = compute_hit_rate(search_results, expected_source)
#     hit_rates.append({"query": query, "hit_rate": hit_rate})

# hit_rate_df = pd.DataFrame(hit_rates)
# display(hit_rate_df)


# # we need a single number to rate the retrieval system
# # the mean hit rate is a good metric to evaluate the retrieval system as a whole

# print(f"Mean Hit Rate: {hit_rate_df['hit_rate'].mean():.4f}")
# print(f"Std-dev Hit Rate: {hit_rate_df['hit_rate'].std():.4f}")

#### Evaluating retrieval on other metrics

In [ ]:
retrieval_scorers = [compute_mrr, compute_ndcg, compute_map, compute_hit_rate, compute_precision, compute_recall, compute_f1_score]
retrieval_evaluation = weave.Evaluation(
    name="Retrieval_Evaluation",
    dataset=eval_samples,
    scorers=retrieval_scorers,
    preprocess_model_input=lambda x: {"query": x["question"], "k":5}
)
retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(retriever))


### Using an LLM as a Retrieval Judge

**ref: https://arxiv.org/pdf/2406.06519**

How do we evaluate if we don't have any ground truth? 

We can use a powerful LLM as a judge to evaluate the retriever. 


In [ ]:
# %load prompts/retrieval_eval.py

RETRIEVAL_EVAL_PROMPT ="""
Given a query and a document excerpt, you must provide a score on an integer scale of 0 to 2 with the following meanings:
    0 = represents that the excerpt is irrelevant to the query,
    1 = represents that the excerpt is somewhat relevant to the query,
    2 = represents that the excerpt is is highly relevant to the query.
    

Important Instruction: Assign category 1 if the excerpt is somewhat related to the query but not completely, category 2 if the excerpt only and entirely refers to the query. If neither of these criteria satisfies the query, give it category 0.


Split this problem into steps:
Consider the underlying intent of the query. Measure how well the content matches a likely intent of the query(M).
Measure how trustworthy the excerpt is (T).
Consider the aspects above and the relative importance of each, and decide on a final score (O). 
Final score must be an integer value only.
Do not provide any code in result. Provide each score in the following JSON format: 
{{"final_score": <integer score without providing any reasoning.>}}

## Examples

Example 1: 
<Query>
How do I programmatically access the human-readable run name?
</Query>
<Document>
If you do not explicitly name your run, a random run name will be assigned to the run to help identify the run in the UI. For instance, random run names will look like "pleasant-flower-4" or "misunderstood-glade-2".

If you'd like to overwrite the run name (like snowy-owl-10) with the run ID (like qvlp96vk) you can use this snippet:

import wandbRetrieval_Evaluation

wandb.init()
wandb.run.name = wandb.run.id
wandb.run.save()

</Document>
{{"final_score": 0}}

Example 2:
<Query>
What are Runs?
</Query>
<Document>
A single unit of computation logged by W&B is called a run. You can think of a W&B run as an atomic element of your whole project. You should initiate a new run when you:
 - Train a model
 - Change a hyperparameter
 - Use a different model
 - Log data or a model as a W&B Artifact
 - Download a W&B Artifact

For example, during a sweep, W&B explores a hyperparameter search space that you specify. Each new hyperparameter combination created by the sweep is implemented and recorded as a unique run. 
</Document>
{{"final_score": 2}}

Example 3:
<Query>
How do I use W&B with Keras ?
</Query>
<Document>
We have added three new callbacks for Keras and TensorFlow users, available from wandb v0.13.4. For the legacy WandbCallback scroll down.
These new callbacks,
 - Adhere to Keras design philosophy
 - Reduce the cognitive load of using a single callback (WandbCallback) for everything
 - Make it easy for Keras users to modify the callback by subclassing it to support their niche use case
</Document>
{{"final_score": 1}}

<Query>
{query}
</Query>

<Document>
{document}
</Document>
"""


In [ ]:
# %load scripts/retrieval_judge.py

import cohere
import os
import weave
import asyncio
import json
from typing import List, Dict, Any
from dotenv import load_dotenv

load_dotenv()


client = cohere.AsyncClient(api_key=os.environ["CO_API_KEY"])

@weave.op()
async def evaluate_retriever_using_llm_judge(query: str, passage: str, eval_prompt: str) -> str:
    response = await client.chat(
        message=eval_prompt.format(query=query, document=passage),
        model="command-r-plus",
        temperature=0.0,
        max_tokens=20,
    )
    return response.text


@weave.op()
async def run_retriever_evaluation_using_llm(
    eval_samples: List[Dict[str, Any]],
    retriever: Any,
    eval_prompt: str
) -> List[Dict[str, Any]]:
    scores = []
    for sample in eval_samples:
        query = sample["question"]
        search_results = retriever.search(query, k=5)
        tasks = []
        for result in search_results:
            tasks.append(evaluate_retriever_using_llm_judge(query, result["text"], eval_prompt))
        sample_scores = await asyncio.gather(*tasks)
        parsed_scores = []
        for score in sample_scores:
            try:
                score = json.loads(score)
                parsed_scores.append(score["final_score"])
            except json.JSONDecodeError:
                parsed_scores.append(None)
        scores.append({"query": query, "scores": parsed_scores})
    return scores


In [ ]:
llm_judge_retrieval_results = asyncio.run(run_retriever_evaluation_using_llm(eval_samples, retriever=retriever, eval_prompt=RETRIEVAL_EVAL_PROMPT))

In [ ]:
# we have the scores for each document
llm_judge_retrieval_results_df = pd.DataFrame(llm_judge_retrieval_results)

# we can compute the reciprocal rank of the first document that is relevant to the query i.e. rated as 2 by our llm judge.
def compute_rank_score(scores: List[int]) -> float:
    rank_score = 0
    for rank, result in enumerate(scores, 1):
        if result == 2:
            rank_score = 1 / rank
            return rank_score
    return rank_score

llm_judge_retrieval_results_df["rank_score"] = llm_judge_retrieval_results_df["scores"].map(compute_rank_score)


display(llm_judge_retrieval_results_df)


print(f"Mean Rank Score: {llm_judge_retrieval_results_df['rank_score'].mean():.4f}")
print(f"Std-dev Rank Score: {llm_judge_retrieval_results_df['rank_score'].std():.4f}")
    

## Evaluating the Response

In [ ]:
# %load scripts/response_generator
import os
from typing import List, Dict
import cohere
import weave

class ResponseGenerator(weave.Model):
    model: str
    prompt: str
    client: cohere.Client = None
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.client = cohere.Client(api_key=os.environ["CO_API_KEY"])

    @weave.op()
    def generate_context(self, context: List[Dict[str, any]]) -> str:
        return [{"source": item['source'], "text": item['text']} for item in context]
    
    @weave.op()
    def generate_response(self, query: str, context: List[Dict[str, any]]) -> str:
        contexts = self.generate_context(context)
        response = self.client.chat(
            preamble=self.prompt,
            message=query,
            model=self.model,
            documents=contexts,
            temperature=0.1,
            max_tokens=2000,
        )
        return response.text

    @weave.op()
    def predict(self, query: str, context: List[Dict[str, any]]):
        return self.generate_response(query, context)


In [ ]:
# %load prompts/initial_system
INITIAL_PROMPT = """
Answer to the following question about W&B. Provide an helful and complete answer based only on the provided documents.
"""


In [ ]:
response_generator = ResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)

In [ ]:
# %load scripts/rag_pipeline
import weave
from typing import Optional, Union
from scripts.retriever import Retriever
from scripts.response_generator import ResponseGenerator


class RAGPipeline(weave.Model):
    retriever: Union[weave.Model, Retriever] = None
    response_generator: Union[weave.Model,ResponseGenerator] = None
    top_k: int = 5

    @weave.op()
    def predict(self, query: str):
        context = self.retriever.predict(query, self.top_k)
        return self.response_generator.predict(query, context)


In [ ]:
rag_pipeline = RAGPipeline(retriever=retriever, response_generator=response_generator, top_k=5)

In [ ]:
# %load scripts/response_eval
import difflib
import Levenshtein
from nltk import word_tokenize
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from rouge import Rouge
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate import meteor
from nltk.corpus import wordnet as wn
import weave
import re
import string

wn.ensure_loaded()


def normalize_text(text: str) -> str:
    """
    Normalize the input text by lowercasing, removing punctuation, and extra whitespace.

    Args:
        text (str): The input text to normalize.

    Returns:
        str: The normalized text.
    """
    # Convert to lowercase
    text = text.lower()
    # Split on punctuation before removing it, ensuring numbers are not split
    text = re.sub(r"[^\w\s\d]", " ", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text


@weave.op()
def compute_diff(model_output: str, answer: str) -> float:
    """
    Compute the similarity ratio between the normalized model output and the expected answer.

    Args:
        model_output (str): The output generated by the model.
        answer (str): The expected answer.

    Returns:
        float: The similarity ratio between the normalized model output and the expected answer.
    """
    norm_output = normalize_text(model_output)
    norm_answer = normalize_text(answer)
    return difflib.SequenceMatcher(None, norm_output, norm_answer).ratio()


@weave.op()
def compute_levenshtein(model_output: str, answer: str) -> float:
    """
    Compute the Levenshtein ratio between the normalized model output and the answer.

    Args:
        model_output (str): The output generated by the model.
        answer (str): The expected answer.

    Returns:
        float: The Levenshtein ratio between the normalized model output and the answer.
    """
    norm_output = normalize_text(model_output)
    norm_answer = normalize_text(answer)
    return Levenshtein.ratio(norm_output, norm_answer)


@weave.op()
def compute_rouge(model_output: str, answer: str) -> float:
    """
    Compute the ROUGE-L F1 score between the normalized model output and the reference answer.

    Args:
        model_output (str): The model's generated output.
        answer (str): The reference answer.

    Returns:
        float: The ROUGE-L F1 score.
    """
    norm_output = normalize_text(model_output)
    norm_answer = normalize_text(answer)
    rouge = Rouge(metrics=["rouge-l"], stats="f")
    scores = rouge.get_scores(norm_output, norm_answer)
    return scores[0]["rouge-l"]["f"]


@weave.op()
def compute_bleu(model_output: str, answer: str) -> float:
    """
    Compute the BLEU score between the normalized model output and the reference answer.

    Args:
        model_output (str): The generated output from the model.
        answer (str): The reference answer.

    Returns:
        float: The BLEU score between the normalized model output and the reference answer.
    """
    chencherry = SmoothingFunction()
    smoothing_function = chencherry.method2

    norm_output = normalize_text(model_output)
    norm_answer = normalize_text(answer)
    reference = word_tokenize(norm_answer)
    candidate = word_tokenize(norm_output)
    score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)
    return score


@weave.op
def compute_meteor(model_output: str, answer: str) -> float:
    """
    Compute the METEOR score between the normalized model output and the reference answer.

    Args:
        model_output (str): The model's generated output.
        answer (str): The reference answer.

    Returns:
        float: The METEOR score rounded to 4 decimal places.
    """
    norm_output = normalize_text(model_output)
    norm_answer = normalize_text(answer)
    reference = word_tokenize(norm_answer)
    candidate = word_tokenize(norm_output)
    meteor_score = round(meteor([candidate], reference), 4)
    return meteor_score


In [ ]:
response_scorers = [compute_diff, compute_levenshtein, compute_rouge, compute_bleu]
response_evaluations = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_samples, 
    scorers=response_scorers, 
    preprocess_model_input=lambda x: {"query": x["question"]})
response_scores = asyncio.run(response_evaluations.evaluate(rag_pipeline))


### Using an LLM as a Response Judge

Some metrics cannot be defined objectively and are particularly useful for more subjective or complex criteria.
We care about correctness, faithfulness, and relevance.

- **Answer Correctness** - Is the generated answer correct compared to the reference and thoroughly answers the user's query?
- **Answer Relevancy** - Is the generated answer relevant and comprehensive?
- **Answer Factfulness** - Is the generated answer factually consistent with the context document?


In [ ]:
# %load prompts/correctness_eval
CORRECTNESS_EVAL_PROMPT ="""
You are a Weight & Biases support expert tasked with evaluating the correctness of answers to questions asked by users to a technical support chatbot. 
You are tasked with judging the correctness of a generated answer based on the user's query, and a reference answer.

You will be given the following information:

<query>
{query}
</query>

<reference_answer>
{reference_answer}
</reference_answer>

<generated_answer>
{generated_answer}
</generated_answer>

Important Instruction: To evaluate the generated answer, follow these steps:

1. Intent Analysis: Consider the underlying intent of the query.
2. Relevance: Check if the generated answer addresses all aspects of the question.
3. Accuracy: Compare the generated answer to the reference answer for completeness and correctness.
4. Trustworthiness: Measure how trustworthy the generated answer is when compared to the reference.

Assign a score on an integer scale of 0 to 2 with the following meanings:
- 0 = The generated answer is incorrect and does not satisfy any of the criteria.
- 1 = The generated answer is partially correct, contains mistakes or is not factually correct.
- 2 = The generated answer is correct, completely answers the query, does not contain any mistakes, and is factually consistent with the reference answer.

After your analysis, provide your verdict in the following JSON format:

{{
    "reason": "<<Provide a brief explanation for your decision here>>",
    "final_score": <<Provide a score as per the above guidelines>>,
    "decision": "<<Provide your final decision here, either 'correct' or 'incorrect'>>"
}}

Here are some examples of correct output:

Example 1:
{{
    "reason": "The generated answer has the exact details as the reference answer and completely answers the user's query.",
    "final_score": 2,
    "decision": "correct"
}}

Example 2:
{{
    "reason": "The generated answer doesn't match the reference answer and deviates from the user's query.",
    "final_score": 0,
    "decision": "incorrect"
}}

Example 3:
{{
    "reason": "The generated answer follows the same steps as the reference answer. However, it significantly misses the user's intent,
    "final_score": 1,
    "decision": "incorrect"
}}

Example 4:
{{
    "reason": "The generated is not factually correct and includes assumptions about code methods completely different from the reference answer",
    "final_score": 0,
    "decision": "incorrect"
}}

Please provide your evaluation based on the given information and format your response according to the specified JSON structure.
"""


In [ ]:
client = cohere.AsyncClient(api_key=os.environ["CO_API_KEY"])

@weave.op()
async def evaluate_correctness_using_llm_judge(question: str, answer: str, model_output: str) -> Dict[str, Any]:
    response = await client.chat(
        message=CORRECTNESS_EVAL_PROMPT.format(query=question, reference_answer=answer, generated_answer=model_output),
        model="command-r-plus",
        temperature=0.0,
        max_tokens=150,
    )
    return json.loads(response.text)


In [ ]:
response_scorers = [evaluate_correctness_using_llm_judge]
correctness_evaluations = weave.Evaluation(
    name="Correctness_Evaluation",
    dataset=eval_samples, 
    scorers=response_scorers, 
    preprocess_model_input=lambda x: {"query": x["question"]})
response_scores = asyncio.run(correctness_evaluations.evaluate(rag_pipeline))


## Exercise

1. Implement the `Relevance` and `Faithfulness` evaluators and evaluate the pipeline on all the dimensions.
2. Generate and share a W&B report with the following sections in the form of tables and charts:
    
    - Summary of the evaluation
    - Retreival Evaluations
        - IR Metrics
        - LLM As a Retrieval Judge Metric
    - Response Evalations
        - Traditional NLP Metrics
        - LLM Judgement Metrics
    - Overall Evalations
    - Conclusion
